In [ ]:
import sys
sys.path.append('../build')

from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.3)).Curve(2)

In [ ]:
u0 = 1/ sqrt( (x+1)**2 + (y+1)**2 + (z+1)**2 )

In [ ]:
# fesL2 = SurfaceL2(mesh, order=1, dual_mapping=True)
# u,v = fesL2.TnT()
fesH1 = H1(mesh, order=2, definedon=mesh.Boundaries(".*"))
uH1,vH1 = fesH1.TnT()
print ("ndof H1 = ", fesH1.ndof)

In [ ]:
D=HypersingularOperator(fesH1, intorder=12, leafsize=40, eta=3., eps=1e-11, 
                                    method="svd", testhmatrix=False)


In [ ]:
gfu = GridFunction(fesH1)
gfu.Set (u0, definedon=mesh.Boundaries(".*"))

mass = BilinearForm(uH1*vH1*ds).Assemble().mat
invmass = mass.Inverse(freedofs=fesH1.FreeDofs())

gf = GridFunction(fesH1)
gf.vec.data = (invmass@D.mat*gfu.vec).Evaluate()
print (Norm(D.mat*gfu.vec))
Draw (gf)